# Expedia Hotel Recommendations Kaggle competition

Peeter Piksarv (piksarv .at. gmail.com)

The latest version of this Jupyter notebook is available at https://github.com/ppik/playdata/tree/master/Kaggle-Expedia

This is my take on that particular Kaggle competition started off using [Dataquest tutorial](https://www.dataquest.io/blog/kaggle-tutorial/) by Vik Paruchuri.

In [79]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

import ml_metrics as metrics

%matplotlib notebook

## Data import
Actually don't need to unpack gzipped cvs files, pandas' `read_csv` can handle those, although it can be slower (Reading 1000000 rows from `train.csv.gz` seems to be about 9% slower than from `train.csv` on my laptop).

Additionally, it's a good idea to specify the data types for each column tho ease the memory requirements. By default pandas detects the following data types:

In [2]:
train = pd.read_csv('data/train.csv.gz', nrows=10)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 24 columns):
date_time                    10 non-null object
site_name                    10 non-null int64
posa_continent               10 non-null int64
user_location_country        10 non-null int64
user_location_region         10 non-null int64
user_location_city           10 non-null int64
orig_destination_distance    6 non-null float64
user_id                      10 non-null int64
is_mobile                    10 non-null int64
is_package                   10 non-null int64
channel                      10 non-null int64
srch_ci                      10 non-null object
srch_co                      10 non-null object
srch_adults_cnt              10 non-null int64
srch_children_cnt            10 non-null int64
srch_rm_cnt                  10 non-null int64
srch_destination_id          10 non-null int64
srch_destination_type_id     10 non-null int64
is_booking                   10 non-null int64
c

According to the specification the data fields are following:

`train.csv`

| Column name               | Description                                   | Data type | Equiv. type | Notes |
|---------------------------|-----------------------------------------------|-----------|-------------|-------|
| date_time                 | Timestamp                                     | string    |             | [1]   |
| site_name                 | ID of the Expedia point of sale               | int       | np.int32    |       |
| posa_continent            | ID of continent associated with site_name     | int       | np.int32    |       |
| user_location_country     | The ID of the country the customer is located | int       | np.int32    |       |
| user_location_region      | The ID of the region the customer is located  | int       | np.int32    |       |
| user_location_city        | The ID of the city the customer is located    | int       | np.int32    |       |
| orig_destination_distance | Physical distance between a hotel and a customer at the time of search. A null means the distance could not be calculated | double | np.float64 | |
| user_id                   | ID of user                                    | int       | np.int32    |       |
| is_mobile                 | 1 when a user connected from a mobile device, 0 otherwise | tinyint | np.uint8 | [2] |
| is_package                | 1 if the click/booking was generated as a part of a package (i.e. combined with a flight), 0 otherwise | int | np.uint8 | [2] |
| channel                   | ID of a marketing channel                     | int       | np.int32    |       |
| srch_ci                   | Checkin date                                  | string    |             | [1]   |
| srch_co                   | Checkout date                                 | string    |             | [1]   |
| srch_adults_cnt           | The number of adults specified in the hotel room | int    | np.int32    |       |
| srch_children_cnt         | The number of (extra occupancy) children specified in the hotel room | int | np.int32 | [4] |
| srch_rm_cnt               | The number of hotel rooms specified in the search | int   | np.int32    | [4]   |
| srch_destination_id       | ID of the destination where the hotel search was performed | int | np.int32 |   |
| srch_destination_type_id  | Type of destination                           | int       | np.int32    |       |
| hotel_continent           | Hotel continent                               | int       | np.int32    |       |
| hotel_country             | Hotel country                                 | int       | np.int32    |       |
| hotel_market              | Hotel market                                  | int       | np.int32    |       |
| is_booking                | 1 if a booking, 0 if a click                  | tinyint   | np.uint8    | [2]   |
| cnt                       | Numer of similar events in the context of the same user session | bigint | np.int64 | |
| hotel_cluster             | ID of a hotel cluster                         | int       | np.int32    |       |

`destinations.csv`

| Column name         | Description                                                | Data type | Equiv. type | Notes |
|---------------------|------------------------------------------------------------|-----------|-------------|-------|
| srch_destination_id | ID of the destination where the hotel search was performed | int       | np.int32    |       |
| d1-d149             | latent description of search regions                       | double    | np.float64  | [3,5] |

### Notes
1. Probably it would be good idea to parse dates while loading data. From date information useful features may include duration of the stay, season/month, how much in advance is the booking made, etc.
2. May use np.bool instead.
3. Single or even half-precision might be enough when starting to take account descriptions of search regions.
4. If taking into account if the column srch_children_cnt or srch_room_cnt it may be worthwhile to simply this first to a boolean values if any number of children and/or rooms was specifien in the hotel room.
5. Maybe the required clustering can be done solely on the base of latent descriptions of search regions.


In [3]:
traincols = ['date_time', 'site_name', 'posa_continent', 'user_location_country',
             'user_location_region', 'user_location_city', 'orig_destination_distance',
             'user_id', 'is_mobile', 'is_package', 'channel', 'srch_ci', 'srch_co',
             'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id',
             'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent',
             'hotel_country', 'hotel_market', 'hotel_cluster']
testcols = ['id', 'date_time', 'site_name', 'posa_continent', 'user_location_country',
            'user_location_region', 'user_location_city', 'orig_destination_distance',
            'user_id', 'is_mobile', 'is_package', 'channel', 'srch_ci', 'srch_co',
            'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id',
            'srch_destination_type_id', 'hotel_continent', 'hotel_country', 'hotel_market']

Finding columns in testcols but not in traincols and vice versa:

In [4]:
[col for col in testcols if col not in traincols]

['id']

In [6]:
[col for col in traincols if col not in testcols]

['is_booking', 'cnt', 'hotel_cluster']

I don't know exactly what data colmuns I will be using eventually but I will define the data types for them here anyway just in case. Looking at the data most of the columns are actually non-negative integers so I can use unsigned integers for the most cases. Usage between uint8, uint32 and others was determined by the min and max values in the test dataset.

In [7]:
def read_csv( filename, cols, nrows=None ):
    datecols = ['date_time', 'srch_ci', 'srch_co']
    dateparser = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S', errors='coerce')

    dtypes = {
        'id': np.uint32,
        'site_name': np.uint8,
        'posa_continent': np.uint8,
        'user_location_country': np.uint16,
        'user_location_region': np.uint16,
        'user_location_city': np.uint16,
        'orig_destination_distance': np.float32,
        'user_id': np.uint32,
        'is_mobile': bool,
        'is_package': bool,
        'channel': np.uint8,
        'srch_adults_cnt': np.uint8,
        'srch_children_cnt': np.uint8,
        'srch_rm_cnt': np.uint8,
        'srch_destination_id': np.uint32,
        'srch_destination_type_id': np.uint8,
        'is_booking': bool,
        'cnt': np.uint64,
        'hotel_continent': np.uint8,
        'hotel_country': np.uint16,
        'hotel_market': np.uint16,
        'hotel_cluster': np.uint8,
    }

    df = pd.read_csv(
        filename,
        nrows=nrows,
        usecols=cols,
        dtype=dtypes, # dtype can also specify datatypes for columns that do not excist in the particular datafile
        parse_dates=[col for col in datecols if col in cols], # columns here must be also in usecols
        date_parser=dateparser,
    )
    return df

In [26]:
train = read_csv('data/train.csv.gz', nrows=None, cols=traincols)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37670293 entries, 0 to 37670292
Data columns (total 24 columns):
date_time                    datetime64[ns]
site_name                    uint8
posa_continent               uint8
user_location_country        uint16
user_location_region         uint16
user_location_city           uint16
orig_destination_distance    float32
user_id                      uint32
is_mobile                    bool
is_package                   bool
channel                      uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              uint8
srch_children_cnt            uint8
srch_rm_cnt                  uint8
srch_destination_id          uint32
srch_destination_type_id     uint8
is_booking                   bool
cnt                          uint64
hotel_continent              uint8
hotel_country                uint16
hotel_market                 uint16
hotel_cluster                uint8
dtypes: boo

With these type definitions the entire training set of 37 million entries takes 2.3 GB of memory.

In [9]:
test = read_csv('data/test.csv.gz', cols=testcols)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2528243 entries, 0 to 2528242
Data columns (total 22 columns):
id                           uint32
date_time                    datetime64[ns]
site_name                    uint8
posa_continent               uint8
user_location_country        uint16
user_location_region         uint16
user_location_city           uint16
orig_destination_distance    float32
user_id                      uint32
is_mobile                    bool
is_package                   bool
channel                      uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt              uint8
srch_children_cnt            uint8
srch_rm_cnt                  uint8
srch_destination_id          uint32
srch_destination_type_id     uint8
hotel_continent              uint8
hotel_country                uint16
hotel_market                 uint16
dtypes: bool(2), datetime64[ns](3), float32(1), uint16(5), uint32(3), uint8(8)
mem

Finding missing values in test data:

In [11]:
test.isnull().sum()

id                                0
date_time                         0
site_name                         0
posa_continent                    0
user_location_country             0
user_location_region              0
user_location_city                0
orig_destination_distance    847461
user_id                           0
is_mobile                         0
is_package                        0
channel                           0
srch_ci                          22
srch_co                          17
srch_adults_cnt                   0
srch_children_cnt                 0
srch_rm_cnt                       0
srch_destination_id               0
srch_destination_type_id          0
hotel_continent                   0
hotel_country                     0
hotel_market                      0
dtype: int64

There are also some dates where the check in date is later than check out date:

In [12]:
(test.srch_ci > test.srch_co).sum()

2184

Checking that all of the user_id-s in test set are contained in training set

In [33]:
test_ids = set(test.user_id.unique())
train_ids = set(train.user_id.unique())
test_ids <= train_ids # issubset

True

However, not all all user_ids that are in training data are in 

In [32]:
len(train_ids - test_ids)

17209

Extract month and year field from the date

In [40]:
train['month'] = train['date_time'].dt.month.astype(np.uint8)
train['year'] = train['date_time'].dt.year.astype(np.uint16)

Pick 10000 users for smaller scale testing

In [57]:
sel_user_ids = sorted(random.sample(train_ids, 10000))
sel_train = train[train.user_id.isin(sel_user_ids)]

Create new test and training sets

In [67]:
t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]

Remove click events from t2 as in original test data.

In [70]:
t2 = t2[t2.is_booking == True]

# Data model building
Starting looking at the most common clusters and their properties.

In [75]:
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)

Predicting `most_common_clusters` for every single row in selected test data.

In [78]:
predictions = [most_common_clusters for i in range(len(t2))]

Calculating Mean Average Precision with `mapk` from `ml_metrics`.

In [85]:
target = [[l] for l in t2['hotel_cluster']]
metrics.mapk(target, predictions, k=5)

0.066013887156459186

--
Peeter Piksarv